In [ ]:
import cv2
import numpy as np
from skimage import measure
import matplotlib.pyplot as plt
from tifffile import imread
import pandas as pd
import os
from pathlib import Path

# Create main output structure
batch_dir = Path("FULL_BATCH_ANALYSIS")
batch_dir.mkdir(exist_ok=True)

# Find ALL converted TIFFs
tiff_files = [f for f in os.listdir('.') if '_C1C2C3.tif' in f]
print(f"🔍 Found {len(tiff_files)} images to process")

results = []

for tiff_file in tiff_files:
    # CLEAN FILENAME (remove _C1C2C3.tif)
    clean_name = tiff_file.replace('_C1C2C3.tif', '').replace('_C1C2C3', '')
    print(f"\n[{len(results)+1}/{len(tiff_files)}] Processing {clean_name}...")
    
    # Create individual folder with ORIGINAL name
    img_dir = batch_dir / clean_name
    img_dir.mkdir(exist_ok=True)
    
    # Load raw data
    img = imread(tiff_file)
    c1_blue_raw = cv2.normalize(img[0], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    c2_green_raw = cv2.normalize(img[1], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    c3_red_raw = cv2.normalize(img[2], None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    
    # =============================================================================
    # STEP 0: Save raw channels
    cv2.imwrite(str(img_dir / 'STEP0_raw_blue.png'), c1_blue_raw)
    cv2.imwrite(str(img_dir / 'STEP0_raw_green.png'), c2_green_raw)
    cv2.imwrite(str(img_dir / 'STEP0_raw_red.png'), c3_red_raw)
    
    # =============================================================================
    # STEP 1: NUCLEI (BLUE DAPI)
    _, blue_thresh = cv2.threshold(c1_blue_raw, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blue_open = cv2.morphologyEx(blue_thresh, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    blue_close = cv2.morphologyEx(blue_open, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
    
    contours, _ = cv2.findContours(blue_close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    nuclei_mask = np.zeros_like(blue_thresh)
    total_nuclei = 0
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 50 < area < 1000:
            cv2.fillPoly(nuclei_mask, [cnt], 255)
            total_nuclei += 1
    
    cv2.imwrite(str(img_dir / 'STEP1_blue_threshold.png'), blue_thresh)
    cv2.imwrite(str(img_dir / 'STEP1_blue_cleaned.png'), blue_close)
    cv2.imwrite(str(img_dir / 'STEP1_nuclei_mask.png'), nuclei_mask)
    
    # =============================================================================
    # STEP 2: LIVE (GREEN in nuclei)
    _, green_thresh = cv2.threshold(c2_green_raw, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    green_in_nuclei = cv2.bitwise_and(green_thresh, nuclei_mask)
    green_clean = cv2.morphologyEx(green_in_nuclei, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    
    contours, _ = cv2.findContours(green_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    live_cells = len([c for c in contours if 30 < cv2.contourArea(c) < 800])
    
    cv2.imwrite(str(img_dir / 'STEP2_green_threshold.png'), green_thresh)
    cv2.imwrite(str(img_dir / 'STEP2_green_in_nuclei.png'), green_in_nuclei)
    cv2.imwrite(str(img_dir / 'STEP2_green_clean.png'), green_clean)
    
    # =============================================================================
    # STEP 3: DEAD (RED in nuclei)
    _, red_thresh = cv2.threshold(c3_red_raw, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    red_in_nuclei = cv2.bitwise_and(red_thresh, nuclei_mask)
    red_clean = cv2.morphologyEx(red_in_nuclei, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
    
    contours, _ = cv2.findContours(red_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    dead_cells = len([c for c in contours if 30 < cv2.contourArea(c) < 800])
    
    cv2.imwrite(str(img_dir / 'STEP3_red_threshold.png'), red_thresh)
    cv2.imwrite(str(img_dir / 'STEP3_red_in_nuclei.png'), red_in_nuclei)
    cv2.imwrite(str(img_dir / 'STEP3_red_clean.png'), red_clean)
    
    # =============================================================================
    # STEP 4: FINAL OVERLAY
    overlay = np.stack([c1_blue_raw, np.zeros_like(c1_blue_raw), c3_red_raw], axis=2)
    all_nuclei_contours = [c for c in contours if 50 < cv2.contourArea(c) < 1000]
    
    for cnt in all_nuclei_contours:
        x,y,w,h = cv2.boundingRect(cnt)
        roi_green = green_in_nuclei[y:y+h, x:x+w]
        roi_red = red_in_nuclei[y:y+h, x:x+w]
        if np.sum(roi_green) > np.sum(roi_red) * 1.5:
            cv2.drawContours(overlay, [cnt], -1, (0, 255, 0), 3)
        elif np.sum(roi_red) > 0:
            cv2.drawContours(overlay, [cnt], -1, (0, 0, 255), 3)
        else:
            cv2.drawContours(overlay, [cnt], -1, (128, 128, 128), 2)
    
    cv2.imwrite(str(img_dir / 'STEP4_final_overlay.png'), overlay)
    
    # Calculate metrics
    viability_pct = (live_cells / total_nuclei * 100) if total_nuclei > 0 else 0
    dead_pct = (dead_cells / total_nuclei * 100) if total_nuclei > 0 else 0
    ambiguous = total_nuclei - live_cells - dead_cells
    
    # Store results
    results.append({
        'Image_Name': clean_name,
        'Total_Cells': total_nuclei,
        'Live_Cells': live_cells,
        'Dead_Cells': dead_cells,
        'Ambiguous_Cells': ambiguous,
        'Viability_%': round(viability_pct, 1),
        'Dead_%': round(dead_pct, 1)
    })
    
    print(f"  ✅ {clean_name}: {live_cells}/{total_nuclei} live ({viability_pct:.1f}%)")

# =============================================================================
# CREATE SUMMARY FILES
df = pd.DataFrame(results)
df = df.sort_values('Viability_%', ascending=False)

# CSV
csv_path = batch_dir / 'cell_viability_summary.csv'
df.to_csv(csv_path, index=False)

# Excel
df.to_excel(batch_dir / 'cell_viability_summary.xlsx', index=False)

print(f"\n🎉 BATCH COMPLETE!")
print(f"📁 Folder structure:")
print(f"   FULL_BATCH_ANALYSIS/")
print(f"   ├── 10_well1_002/          ← ALL 14 STEP images")
print(f"   ├── 30_well3_001/          ← ALL 14 STEP images") 
print(f"   ├── 70_well1_001/          ← ALL 14 STEP images")
print(f"   ├── cell_viability_summary.csv")
print(f"   └── cell_viability_summary.xlsx")

print(f"\n📊 SUMMARY PREVIEW:")
print(df[['Image_Name', 'Total_Cells', 'Live_Cells', 'Dead_Cells', 'Viability_%']].round(1).to_string(index=False))
